In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
import pandas as pd

# Introduction to Linear Regression

**OBJECTIVES**

- Derive ordinary least squares models for data
- Evaluate regression models using mean squared error
- Examine errors and assumptions in least squares models
- Use `scikit-learn` to fit regression models to data

## Calculus Refresher

An important idea is that of finding a maximum or minimum of a function.  From calculus, we have the tools required.  Specifically, a maximum or minimum value of a function $f$ occurs wherever $f'(x) = 0$ or is underfined.  Consider the function:

$$f(x) = x^2$$

In [ ]:
def f(x): return x**2
x = np.linspace(-2, 2, 100)
plt.plot(x, f(x))
plt.grid()
plt.title(r'$f(x) = x^2$');

Here, using our power rule for derivatives of polynomials we have:

$$f'(x) = 2x$$

and are left to solve:

$$0 = 2x$$

or 

$$x = 0$$

**PROBLEM**: Determine where the function $f(x) = (5 - 2x)^2$ has a minimum.

In [ ]:
def f(x): return (5 - 2*x)**2
x = np.linspace(-2, 5, 100)
plt.plot(x, f(x))
plt.grid();

#### Using the chain rule

**Example 1**: Line of best fit

In [ ]:
import seaborn as sns

In [ ]:
tips = sns.load_dataset('tips')

In [ ]:
tips.head()

In [ ]:
sns.scatterplot(data = tips, x = 'total_bill', y = 'tip');

In [ ]:
def y1(x): return .19*x

In [ ]:
def y2(x): return .12*x

In [ ]:
x = tips['total_bill']

In [ ]:
plt.plot(x, y1(x), label = 'y1')
plt.plot(x, y2(x), label = 'y2')
plt.legend()
sns.scatterplot(data = tips, x = 'total_bill', y = 'tip')
plt.title('How to determine the line of best fit?')
plt.grid();

To decide between all possible lines we will examine the error in all these models and select the one that minimizes this error.

In [ ]:
plt.plot(x, y2(x))
for i, yhat in enumerate(y2(x)):
    plt.vlines(x = tips['total_bill'].iloc[i], ymin = yhat, ymax = tips['tip'].iloc[i], color = 'red', linestyle = '--')
sns.scatterplot(data = tips, x = 'total_bill', y = 'tip')
plt.title('Error in Model')
plt.grid();

#### Mean Squared Error

$$\text{MSE}(\beta_0) = \frac{1}{n}\sum_{i = 1}^n (y_i - \beta_0x)^2$$



**OBJECTIVE**: Minimize mean squared error

In [ ]:
def mse(beta):
    return np.mean((y - beta*x)**2)

In [ ]:
x = tips['total_bill']
y = tips['tip']

In [ ]:
mse(.17)

In [ ]:
for pct in np.linspace(.1, .2, 11):
    print(f'The MSE for slope {pct: .3f} is {mse(pct): .3f}')

In [ ]:
betas = np.linspace(0, .4, 100)
plt.plot(betas, [mse(beta) for beta in betas])
plt.xlabel(r'$\beta$')
plt.ylabel('MSE')
plt.title('Mean Squared Error for Different Slopes')
plt.grid();

#### Using `scipy` 

To find the minimum of our objective function, the `minimize` function from `scipy.optimize` is useful.  This relies on a variety of different optimization algorithms to find the minimum of a function.

In [ ]:
from scipy.optimize import minimize

In [ ]:
#function to minimize and starting guess
minimize(mse, .1 )

### Solving the Problem Exactly

From calculus we know that the minimum value for a quadratic will occur where the first derivative equals zero.  Thus, to determine the equations for the line of best fit, we minimize the MSE function with respect to $\beta$.

$$f(\beta) = \frac{1}{n}\sum_{i = 1}^n (y - \beta x)^2$$

$$f'(\beta) = \frac{-2}{n}\sum_{i = 1}^n(y - \beta x) x$$

$$ 0 = \frac{-2}{n}\sum_{i = 1}^n(y - \beta x) x$$

$$0 = \sum_{i = 1}^n(y - \beta x) x$$

$$0 = \sum yx - \sum \beta x^2 $$

$$\sum \beta x^2 = \sum y x$$

$$\beta \sum x^2 = \sum y x$$

$$\beta = \frac{\sum y x}{\sum x^2}$$

In [ ]:
np.sum(tips['total_bill']*tips['tip'])/np.sum(tips['total_bill']**2)

### Adding an intercept

Consider the model:

$$y = \beta_0 + \beta_1 x + \epsilon$$

where $\epsilon = N(0, 1)$.

In [ ]:
np.random.seed(42)
x = np.linspace(0, 3, 40)
y = 3*x + 4 + np.random.normal(size = len(x))
plt.scatter(x, y)

Now, the objective function changes to be a function in 3-Dimensions where the slope and intercept terms are input and mean squared error is the output.

$$\text{MSE}(\beta_0, \beta_1) = \frac{1}{n}\sum_{i = 1}^n (y - (\beta_0 + \beta_1 x)^2)$$

In [ ]:
def mse(betas):
    return np.mean((y - (betas[0] + betas[1]*x))**2)

In [ ]:
mse([4, 3])

In [ ]:
minimize(mse, [0, 0])

In [ ]:
betas = minimize(mse, [0, 0]).x

In [ ]:
def lobf(x): return betas[0] + betas[1]*x

In [ ]:
plt.scatter(x, y)
plt.plot(x, lobf(x), color = 'red')
plt.grid()
plt.title('Line of Best fit with slope and intercept');

### Exercise

Use the `minimize` function together with your `mse` function to complete the class below.  After calling the fit method assign the slope of the line of best fit to the `.coef_` attribute and the $y$-intercept to the `.intercept_` attribute.

Test your model on the tips data below.

In [ ]:
class LinearReg:
    '''
    This class fits an ordinary lease squares model
    of the form beta_0 + beta_1 * x.
    '''
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
        
    def mse(self, betas):
        return np.mean((y - (betas[0] + betas[1]*x))**2)
        
    def fit(self, x, y):
        #betas that minimize
        betas = minimize(self.mse, [0, 0]).x
        #set coef_
        self.coef_ = betas[1]
        #set intercept_
        self.intercept_ = betas[0]
        
    
    def predict(self, x):
        return self.intercept_ + self.coef_*x
    
    

In [ ]:
x = tips['total_bill']
y = tips['tip']

In [ ]:
#instantiate the model
model = LinearReg()

In [ ]:
#fit the model on data
model.fit(x, y)

In [ ]:
#make predictions on all data
model.predict(x)

In [ ]:
model.coef_

In [ ]:
model.intercept_

#### A second example

In [ ]:
import statsmodels.api as sm
duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData")
print(duncan_prestige.__doc__)

In [ ]:
sns.lmplot(duncan_prestige.data, x = 'education', y = 'prestige')
plt.title('Education vs. Prestige');

- Fit a model with an intercept to the data.  
- What is the slope of the line and what does this mean in terms of education and income?
- What is the intercept of the model and what does this mean in terms of education and income?

In [ ]:
x = duncan_prestige.data[['education']]
y = duncan_prestige.data['prestige']

In [ ]:
x.shape

In [ ]:
# !pip install -U scikit-learn

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
#instantiate -- step 1

In [ ]:
#fit it


In [ ]:
#make predictions

In [ ]:
#slope and intercept

#### Examining Errors in the Model

Once we have a model, it is important to examine the properties of the residuals.  Specifically, we aim to examine the residuals for patterns in error and the overall distribution of these errors.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (20, 10))
sns.residplot(x = x, y = y, ax = ax[0], lowess=True)
ax[0].set_title('Scatterplot of Residuals')
sns.kdeplot((y - (model.intercept_ + model.coef_[0]*x.values[:, 0])), ax = ax[1], fill = True);
ax[1].grid()
ax[1].set_title('Distribution of residuals');

### Using `scikit-learn`

The `scikit-learn` library has many predictive models and modeling tools.  It is a popular library in industry for Machine Learning tasks. [docs](https://scikit-learn.org/stable/index.html)

In [ ]:
# !pip install -U scikit-learn

In [ ]:
credit = pd.read_csv('data/Credit.csv', index_col=0)
credit.head()

In [ ]:
from sklearn.linear_model import LinearRegression

**PROBLEM**: Which feature is the strongest predictor of `Balance` in the data?

In [ ]:
sns.heatmap(credit.corr(numeric_only = True), annot = True)

In [ ]:
X = credit[['Rating']]
y = credit['Balance']

**PROBLEM**: Build a `LinearRegression` model, determine the **Root Mean Squared Error** and interpret the slope and intercept.

In [ ]:
credit_model = LinearRegression()

In [ ]:
credit_model.fit(X, y)

In [ ]:
print(f'The model is y = {credit_model.coef_[0]}x + {credit_model.intercept_}')

**PROBLEM**: Examine the residual plot and histogram of residuals.  Do they look as they should?

In [ ]:
sns.residplot(data = credit, x = X, y = y)

In [ ]:
plt.hist(y - credit_model.predict(X))

#### Other Models

If your goal is more around statistical inference and you want information about things like hypothesis tests on coefficients, the `statsmodels` library is a more classical statistics interface that also contains a variety of regression models.  Below, the `OLS` model is instantiated, fit, and the results summarized with the `.summary()` method.

In [ ]:
import statsmodels.api as sm

In [ ]:
#instantiate the model


In [ ]:
#create the intercept term


In [ ]:
#fit the model


In [ ]:
#summary


Typically, we will use the `scikitlearn` models on our data.  After break will focus on regression models with more features and next class explore how we can build higher degree polynomial models for our data.